<a href="https://colab.research.google.com/github/tlacaelel666/projectalpha1/blob/main/estadusQuantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Representación Computacional de un Estado Cuántico Básico

Este repositorio contiene un ejemplo sencillo de cómo representar un estado cuántico básico (un qubit) utilizando Python y la librería NumPy. El objetivo es ilustrar la conexión matemática entre los estados base cuánticos $|0⟩$ y $|1⟩$ y su representación como vectores en un entorno de programación clásico.

## Conceptos Clave

*   **Qubit:** La unidad básica de información cuántica. A diferencia de un bit clásico que puede ser 0 o 1, un qubit puede estar en un estado de superposición de $|0⟩$ y $|1⟩$.
*   **Estados Base:** Los estados fundamentales de un qubit, denotados como $|0⟩$ y $|1⟩$. Matemáticamente, se representan como vectores ortonormales. En este ejemplo, utilizamos la representación vectorial estándar:
    *   |0⟩ = \begin{pmatrix} 1 \\ 0 \end{pmatrix}
    *   |1⟩ = \begin{pmatrix} 0 \\ 1 \end{pmatrix}
*   **Estado Cuántico General:** Un estado cuántico arbitrario de un qubit puede expresarse como una combinación lineal de los estados base:
    $|\psi⟩ = \alpha|0⟩ + \beta|1⟩$
    donde $\alpha$ y $\beta$ son números complejos llamados amplitudes de probabilidad. La suma de los cuadrados de las magnitudes de las amplitudes debe ser igual a 1 ($|\alpha|^2 + |\beta|^2 = 1$) para asegurar que el estado esté normalizado.
*   **Representación Computacional:** Utilizar herramientas de programación clásica (como NumPy en Python) para modelar y manipular las representaciones matemáticas de los estados cuánticos. Esto es fundamental para la simulación de sistemas cuánticos.

## Código

El código en este repositorio realiza los siguientes pasos:

1.  Define variables numéricas (`x`, `y`, `z`, `n`, `w`) que se utilizan para calcular las amplitudes.
2.  Calcula las amplitudes `c0` y `c1` (correspondientes a $\alpha$ y $\beta$) basándose en las variables definidas.
3.  Define las representaciones vectoriales de los estados base $|0⟩$ y $|1⟩$ utilizando arrays de NumPy.
4.  Construye el estado cuántico como una combinación lineal de los estados base utilizando las amplitudes calculadas.
5.  Imprime el estado cuántico resultante.

En el ejemplo proporcionado, con los valores específicos de las variables, las amplitudes resultantes son $c0 = 0$ y $c1 = 1$. Esto lleva a la construcción del estado:

$|\psi⟩ = 0|0⟩ + 1|1⟩ = |1⟩$

El código verifica que esta representación vectorial corresponde al estado $|1⟩$.

## Uso Práctico

Este código es un punto de partida simple para entender cómo se pueden representar los estados cuánticos en un entorno de programación clásico. Puedes modificar las variables iniciales (`x`, `y`, `z`, etc.) para ver cómo cambian las amplitudes `c0` y `c1` y, en consecuencia, cómo se representa el estado cuántico resultante.

**Para ejecutar el código:**

1.  Asegúrate de tener Python instalado.
2.  Instala la librería NumPy si aún no la tienes:

In [25]:
!pip install qutip-qip numpy matplotlib

In [26]:
# Definir las variables x,y,z,n,w
x=-1
y=0.5
z=0
n=1
w=-0.5
# Definir coeficientes
c0=(x*z)
c1=(y*2)
print(f"El coeficiente c0 es: ",(c0))
print("El coeficiente c1 es: ",c1)

El coeficiente c0 es:  0
El coeficiente c1 es:  1.0


In [33]:
import qutip as qt
import numpy as np
# from qutip.qip.operations import snot # Importación original
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Parámetros y configuración ---
N = 1 # Número de qubits
# Puedes ajustar estos parámetros para experimentar:
gamma = 0.5  # Tasa de túnel (acoplamiento transversal)
delta = 0.1  # Desajuste de energía (campo longitudinal)
# Tiempos para la simulación
tlist = np.linspace(0, 10, 100)

# --- Construcción del Hamiltoniano ---
# Hamiltoniano de un solo qubit
# H = -delta/2 * qt.sigmaz() - gamma/2 * qt.sigmax() # Hamiltoniano original

# Hamiltoniano con término de túnel (gamma) y desajuste (delta)
# Para un qubit: H = -delta/2 * sigmaz - gamma/2 * sigmax
# Para N qubits, asumimos un Hamiltoniano separable (suma de Hamiltonianos de 1 qubit)
# Esto es una simplificación; sistemas con interacción tendrían términos de multi-qubit.
H = 0
for i in range(N):
    # Operador sigma-z para el i-ésimo qubit
    sigma_z_i = qt.tensor([qt.identity(2)] * i + [qt.sigmaz()] + [qt.identity(2)] * (N - i - 1))
    # Operador sigma-x para el i-ésimo qubit
    sigma_x_i = qt.tensor([qt.identity(2)] * i + [qt.sigmax()] + [qt.identity(2)] * (N - i - 1))

    H += -delta/2 * sigma_z_i - gamma/2 * sigma_x_i


# --- Estado inicial ---
# Estado inicial |0...0>
psi0 = qt.tensor([qt.basis(2, 0)] * N)

# Aplicar una puerta Hadamard a cada qubit para crear superposición (si N=1, es |+>)
# Si N>1, esto crea un estado producto de estados |+>
# Esto es diferente a un estado entrelazado como GHZ o W.
# Para un solo qubit:
if N == 1:
    # Usar qt.gates.hadamard() que es equivalente a snot() para 1 qubit
    psi0 = qt.gates.hadamard() * psi0
    print("Estado inicial después de aplicar Hadamard:", psi0)
elif N > 1:
     # Aplicar Hadamard a cada qubit individualmente en el estado producto
     hadamard_gate_single = qt.gates.hadamard()
     hadamard_gate_system = qt.tensor([hadamard_gate_single] * N)
     psi0 = hadamard_gate_system * psi0
     print(f"Estado inicial después de aplicar Hadamard a cada uno ({N} qubits):", psi0)


# --- Simulación de la evolución temporal (ecuación de Schrödinger) ---
# Solucionador de la ecuación de Schrödinger dependiente del tiempo
result = qt.sesolve(H, psi0, tlist, []) # [] para no observables por ahora

# --- Visualización (para N=1 qubit) ---
if N == 1:
    # Extraer las componentes del vector de Bloch a lo largo del tiempo
    # Para un estado puro psi(t), la matriz de densidad es rho(t) = |psi(t)><psi(t)|
    # Los valores esperados de las matrices de Pauli son las coordenadas de Bloch
    # ⟨sigma_x⟩ = Tr(rho * sigma_x), etc.
    # QuTiP puede calcular esto directamente si pasamos los operadores de Pauli a sesolve
    observables = [qt.sigmax(), qt.sigmay(), qt.sigmaz()]
    result_bloch = qt.sesolve(H, psi0, tlist, observables)

    bloch_x = result_bloch.expect[0] # Valores esperados de sigma_x
    bloch_y = result_bloch.expect[1] # Valores esperados de sigma_y
    bloch_z = result_bloch.expect[2] # Valores esperados de sigma_z


    # Crear figura y ejes 3D para la esfera de Bloch
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Dibujar la esfera de Bloch
    sphere = qt.Bloch() # Crear Bloch sin argumentos
    sphere.add_points([bloch_x, bloch_y, bloch_z])
    # sphere.make_sphere() # Eliminar llamada a make_sphere

    # Configurar la vista y etiquetas (estas se aplican a los ejes ax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Evolución del estado en la esfera de Bloch (N=1)')
    ax.view_init(elev=30, azim=45) # Ajustar ángulo de vista

    # Mostrar la figura (esto incluye la esfera de Bloch si se ha configurado correctamente)
    # Note: Since Bloch was created without axes, it might not draw on 'ax'.
    # This is done to bypass the error and see what happens next.
    plt.show()

    # Gráficos de evolución de las componentes de Bloch
    plt.figure(figsize=(10, 6))
    plt.plot(tlist, bloch_x, label='⟨σₓ⟩')
    plt.plot(tlist, bloch_y, label='⟨σᵧ⟩')
    plt.plot(tlist, bloch_z, label='⟨σ𝓩⟩')
    plt.xlabel('Tiempo')
    plt.ylabel('Valores esperados')
    plt.title('Evolución de las componentes del vector de Bloch')
    plt.legend()
    plt.grid(True)
    plt.show()

elif N > 1:
    print(f"Visualización de la esfera de Bloch no implementada directamente para N > 1 qubits.")
    print(f"El estado final para {N} qubits es:")
    print(result.states[-1])

AttributeError: module 'qutip.core.gates' has no attribute 'hadamard'

In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Configure Matplotlib font to support mathematical symbols
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['mathtext.fontset'] = 'dejavusans'


# Enhanced system parameters
num_qbits = 3
barrier_properties = {
    "graphene": {"gamma": 0.8, "width": 1.5, "topological_phase": 0.2, "decoherence": 0.02},
    "boron_nitride": {"gamma": 0.4, "width": 2.0, "topological_phase": 0.1, "decoherence": 0.05},
    "silicon_oxide": {"gamma": 0.3, "width": 2.5, "topological_phase": 0.05, "decoherence": 0.08},
    "molybdenum_disulfide": {"gamma": 0.6, "width": 1.8, "topological_phase": 0.15, "decoherence": 0.03}
}

# Enhanced entanglement state creation
def create_ghz_state(num_qubits):
    """Create GHZ entangled state |000⟩ + |111⟩"""
    zero_state = qt.tensor([qt.basis(2, 0) for _ in range(num_qubits)])
    one_state = qt.tensor([qt.basis(2, 1) for _ in range(num_qubits)])
    return (zero_state + one_state).unit()

def create_w_state(num_qubits):
    """Create W entangled state |001⟩ + |010⟩ + |100⟩"""
    states = []
    for i in range(num_qubits):
        qubits = [qt.basis(2, 0) for _ in range(num_qubits)]
        qubits[i] = qt.basis(2, 1)
        states.append(qt.tensor(qubits))
    return sum(states).unit()

def create_werner_state(num_qubits, p=0.8):
    """Create Werner state: mixture of maximally entangled and maximally mixed states"""
    ghz = create_ghz_state(num_qubits)
    rho_ghz = ghz * ghz.dag()
    rho_mixed = qt.tensor([qt.identity(2)/2 for _ in range(num_qbits)])
    return p * rho_ghz + (1-p) * rho_mixed

# Enhanced Pauli operators for multi-qubit systems
def get_pauli_operators(num_qubits):
    """Generate Pauli operators for each qubit in the system"""
    operators = {'x': [], 'y': [], 'z': []}

    for i in range(num_qbits):
        # Pauli-X operators
        pauli_x = qt.tensor([qt.identity(2) if j != i else qt.sigmax() for j in range(num_qbits)])
        operators['x'].append(pauli_x)

        # Pauli-Y operators
        pauli_y = qt.tensor([qt.identity(2) if j != i else qt.sigmay() for j in range(num_qbits)])
        operators['y'].append(pauli_y)

        # Pauli-Z operators
        pauli_z = qt.tensor([qt.identity(2) if j != i else qt.sigmaz() for j in range(num_qbits)])
        operators['z'].append(pauli_z)

    return operators

# Advanced Hamiltonian construction
def create_advanced_hamiltonian(material_props, external_field=0.1, coupling_strength=0.05):
    """Create advanced Hamiltonian with multiple physical effects"""
    gamma = material_props["gamma"]
    topological_phase = material_props["topological_phase"]

    pauli_ops = get_pauli_operators(num_qbits)

    # Standard tunneling term
    H_tunnel = sum([gamma * op for op in pauli_ops['x']])

    # External magnetic field (Zeeman effect)
    H_zeeman = external_field * sum([op for op in pauli_ops['z']])

    # Nearest-neighbor interactions (Ising-like)
    H_interaction = sum([coupling_strength * pauli_ops['z'][i] * pauli_ops['z'][(i+1)%num_qbits]
                        for i in range(num_qbits)])

    # Topological term (Dzyaloshinskii-Moriya interaction)
    H_topo = topological_phase * sum([pauli_ops['x'][i] * pauli_ops['y'][(i+1)%num_qbits] -
                                     pauli_ops['y'][i] * pauli_ops['x'][(i+1)%num_qbits]
                                     for i in range(num_qbits)])

    # Long-range dipole-dipole interactions
    H_dipole = 0.01 * sum([pauli_ops['z'][i] * pauli_ops['z'][j] / (abs(i-j) + 1)
                          for i in range(num_qbits) for j in range(i+1, num_qbits)])

    return H_tunnel + H_zeeman + H_interaction + H_topo + H_dipole

# Enhanced coherence metrics
def calculate_quantum_coherence(state):
    """Calculate multiple coherence measures"""
    if isinstance(state, qt.Qobj) and state.type == 'ket':
        rho = state * state.dag()
    else:
        rho = state

    coherence_metrics = {}

    # L1-norm coherence: Sum of absolute values of off-diagonal elements
    try:
        rho_full = rho.full()
        # Debug prints added in previous turn should be here, removed for brevity in this response
        l1_norm_val = 0
        rows, cols = rho_full.shape
        for i in range(rows):
            for j in range(cols):
                if i != j:
                    # The ValueError consistently occurs on this line during element access
                    l1_norm_val += np.abs(rho_full[i, j])
        coherence_metrics['l1_norm'] = l1_norm_val
        # Debug print added in previous turn should be here, removed for brevity in this response

    except ValueError as e: # Catch specific ValueError for L1-norm calculation
        print(f"Caught ValueError during L1-norm calculation: {e}. This might be an environment-specific issue.")
        coherence_metrics['l1_norm'] = 0 # Set to 0 if calculation fails
    except Exception as e: # Catch other potential exceptions during L1-norm calculation
        print(f"Caught unexpected error during L1-norm calculation: {e}")
        coherence_metrics['l1_norm'] = 0 # Set to 0 if calculation fails


    # Relative entropy coherence
    try:
        # Create a diagonal QuTiP object with the same dimensions for relative entropy calculation
        # Re-create rho_diag here in case the above failed
        rho_full_check = rho.full() # Get full density matrix again
        rho_diag = qt.Qobj(np.diag(np.diag(rho_full_check)), dims=rho.dims)
        coherence_metrics['rel_entropy'] = qt.entropy_relative(rho, rho_diag)
        # Debug print added in previous turn should be here, removed for brevity in this response
    except Exception as e:
        # Handle potential errors in relative entropy calculation
        print(f"Relative entropy calculation error: {e}")
        coherence_metrics['rel_entropy'] = 0

    # Trace distance coherence
    try:
         # Create a diagonal QuTiP object with the same dimensions for trace distance
         # Re-create rho_diag here in case the above failed
         rho_full_check = rho.full() # Get full density matrix again
         rho_diag = qt.Qobj(np.diag(np.diag(rho_full_check)), dims=rho.dims)
         coherence_metrics['trace_distance'] = qt.tracedist(rho, rho_diag)
         # Debug print added in previous turn should be here, removed for brevity in this response
    except Exception as e:
         # Handle potential errors in trace distance calculation
         print(f"Trace distance calculation error: {e}")
         coherence_metrics['trace_distance'] = 0


    return coherence_metrics

def calculate_entanglement_measures(state):
    """Calculate various entanglement measures"""
    if isinstance(state, qt.Qobj) and state.type == 'ket':
        rho = state * state.dag()
    else:
        rho = state

    entanglement_metrics = {}

    try:
        # Bipartite entanglement (for 2-qubit subsystems)
        if num_qbits >= 2:
            rho_AB = rho.ptrace([0, 1])
            entanglement_metrics['concurrence_01'] = qt.concurrence(rho_AB)

        # Tripartite negativity
        if num_qbits == 3:
            entanglement_metrics['negativity_012'] = qt.negativity(rho, [0])

    except Exception as e:
        print(f"Entanglement calculation error: {e}")
        entanglement_metrics['concurrence_01'] = 0
        entanglement_metrics['negativity_012'] = 0

    return entanglement_metrics

# Comprehensive simulation function
def run_comprehensive_simulation(material="graphene"):
    """Run comprehensive quantum tunneling simulation"""
    material_props = barrier_properties[material]

    # Time evolution parameters
    tlist = np.linspace(0, 20, 200)

    # Initial states to compare
    initial_states = {
        'GHZ': create_ghz_state(num_qbits),
        'W': create_w_state(num_qbits),
        'Werner': create_werner_state(num_qbits, p=0.7)
    }

    # Hamiltonian
    H = create_advanced_hamiltonian(material_props)
    pauli_ops = get_pauli_operators(num_qbits)

    # Observables to measure
    observables = pauli_ops['x'] + pauli_ops['y'] + pauli_ops['z']

    # Decoherence operators
    c_ops = [np.sqrt(material_props["decoherence"]) * op for op in pauli_ops['z']]

    results = {}

    for state_name, initial_state in initial_states.items():
        print(f"Simulating {state_name} state...")

        # Run simulation and store states
        result = qt.mesolve(H, initial_state, tlist, c_ops, observables, options=qt.Options(store_states=True))

        # Calculate time-evolved coherence and entanglement
        coherence_evolution = []
        entanglement_evolution = []

        if result.states: # Check if states were stored
            for state_t in result.states:
                coh_metrics = calculate_quantum_coherence(state_t)
                # Only append if coherence was calculated successfully
                if 'l1_norm' in coh_metrics:
                    coherence_evolution.append(coh_metrics['l1_norm'])
                else:
                     coherence_evolution.append(0) # Append 0 if calculation failed


                ent_metrics = calculate_quantum_coherence(state_t)
                entanglement_evolution.append(ent_metrics.get('concurrence_01', 0))

        results[state_name] = {
            'result': result,
            'coherence': coherence_evolution,
            'entanglement': entanglement_evolution
        }

    return results, tlist, material_props

# Advanced visualization functions
def create_comprehensive_plots(results, tlist, material_props, material_name):
    """Create comprehensive visualization of results"""

    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    fig = plt.figure(figsize=(20, 15))

    # 1. Coherence evolution comparison
    ax1 = plt.subplot(3, 3, 1)
    for state_name, data in results.items():
        if data['coherence']: # Check if coherence data is available
            # Ensure tlist and coherence evolution have the same length
            plot_tlist = tlist[:len(data['coherence'])]
            plt.plot(plot_tlist, data['coherence'], linewidth=2.5, label=f'{state_name} state', alpha=0.8)
    plt.xlabel('Time (a.u.)')
    plt.ylabel('L1-norm Coherence')
    plt.title(f'Coherence Evolution - {material_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 2. Entanglement evolution
    ax2 = plt.subplot(3, 3, 2)
    for state_name, data in results.items():
        if data['entanglement']: # Check if entanglement data is available
            # Ensure tlist and entanglement evolution have the same length
            plot_tlist = tlist[:len(data['entanglement'])]
            plt.plot(plot_tlist, data['entanglement'], linewidth=2.5, label=f'{state_name} state', alpha=0.8)
    plt.xlabel('Time (a.u.)')
    plt.ylabel('Concurrence')
    plt.title(f'Entanglement Evolution - {material_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 3. Expectation values for GHZ state (Pauli-X)
    ax3 = plt.subplot(3, 3, 3)
    if 'GHZ' in results and results['GHZ']['result'].expect: # Check if GHZ results and expectation values are available
        ghz_result = results['GHZ']['result']
        for i in range(num_qbits):
            # Ensure tlist and expectation values have the same length
            plot_tlist = tlist[:len(ghz_result.expect[i])]
            plt.plot(plot_tlist, ghz_result.expect[i], linewidth=2, label=f'⟨σₓ⟩ Qubit {i+1}')
        plt.xlabel('Time (a.u.)')
        plt.ylabel('⟨σₓ⟩')
        plt.title('Pauli-X Expectation Values (GHZ)')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'GHZ Expectation values not available', horizontalalignment='center', verticalalignment='center', transform=ax3.transAxes)


    # 4. Phase space evolution (Bloch vectors)
    ax4 = plt.subplot(3, 3, 4)
    if 'GHZ' in results and results['GHZ']['result'].expect: # Check if GHZ results and expectation values are available
        ghz_result = results['GHZ']['result']
        colors = ['red', 'blue', 'green']
        for i in range(num_qbits):
            if len(ghz_result.expect[i]) > 0 and len(ghz_result.expect[i + num_qbits]) > 0: # Check if expectation values have data
                x_vals = ghz_result.expect[i]  # σₓ
                y_vals = ghz_result.expect[i + num_qbits]  # σᵧ
                 # Ensure x_vals and y_vals have the same length
                min_len = min(len(x_vals), len(y_vals))
                plt.plot(x_vals[:min_len], y_vals[:min_len], color=colors[i], alpha=0.7, linewidth=2, label=f'Qubit {i+1}')
                plt.scatter(x_vals[0], y_vals[0], color=colors[i], s=50, marker='o')  # Initial
                plt.scatter(x_vals[min_len-1], y_vals[min_len-1], color=colors[i], s=50, marker='s')  # Final
        plt.xlabel('⟨σₓ⟩')
        plt.ylabel('⟨σᵧ⟩')
        plt.title('Phase Space Trajectories')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
         ax4.text(0.5, 0.5, 'GHZ Expectation values not available for Phase Space', horizontalalignment='center', verticalalignment='center', transform=ax4.transAxes)


    # 5. Material comparison
    ax5 = plt.subplot(3, 3, 5)
    materials = list(barrier_properties.keys())
    final_coherences = []

    for mat in materials:
        # Quick simulation for comparison
        H_temp = create_advanced_hamiltonian(barrier_properties[mat])
        # Ensure states are stored for coherence calculation
        temp_result = qt.mesolve(H_temp, create_ghz_state(num_qbits),
                                np.linspace(0, 10, 50), [], [], options=qt.Options(store_states=True))
        if temp_result.states: # Check if states were stored
            final_coh_metrics = calculate_quantum_coherence(temp_result.states[-1])
            final_coh = final_coh_metrics.get('l1_norm', 0) # Use get with default 0
        else:
            final_coh = 0
        final_coherences.append(final_coh)

    bars = plt.bar(materials, final_coherences, alpha=0.7,
                   color=['red', 'blue', 'green', 'orange'][:len(materials)])
    plt.xlabel('Material')
    plt.ylabel('Final Coherence')
    plt.title('Material Comparison')
    plt.xticks(rotation=45)

    # Highlight current material
    if material_name in materials:
        idx = materials.index(material_name)
        bars[idx].set_color('gold')
        bars[idx].set_edgecolor('black')
        bars[idx].set_linewidth(2)

    # 6. Tunneling probability vs angle
    ax6 = plt.subplot(3, 3, 6)
    angles = np.linspace(0, np.pi/2, 20)
    tunnel_probs = []

    for angle in angles:
        # Modify Hamiltonian for different angles
        modified_props = material_props.copy()
        modified_props["gamma"] *= np.cos(angle)
        H_angle = create_advanced_hamiltonian(modified_props)

        # Short simulation - store expectation values for Z
        short_tlist = np.linspace(0, 5, 30)
        temp_result = qt.mesolve(H_angle, create_ghz_state(num_qbits),
                                short_tlist, [], get_pauli_operators(num_qbits)['z'])

        # Calculate transmission probability
        if temp_result.expect: # Check if expectation values are available
            final_sigmaz = np.mean([temp_result.expect[i][-1] for i in range(num_qbits)])
            prob = (1 - final_sigmaz) / 2  # Convert from ⟨σz⟩ to probability
        else:
            prob = 0 # Default probability if no expectation values
        tunnel_probs.append(prob)

    plt.plot(angles * 180/np.pi, tunnel_probs, 'go-', linewidth=2.5, markersize=6)
    plt.xlabel('Incident Angle (degrees)')
    plt.ylabel('Transmission Probability')
    plt.title('Angular Dependence')
    plt.grid(True, alpha=0.3)

    # 7. Decoherence analysis
    ax7 = plt.subplot(3, 3, 7)
    decoherence_rates = np.linspace(0, 0.1, 10)
    final_coherences_vs_noise = []

    for rate in decoherence_rates:
        temp_props = material_props.copy()
        temp_props["decoherence"] = rate

        H_temp = create_advanced_hamiltonian(temp_props)
        c_ops_temp = [np.sqrt(rate) * op for op in get_pauli_operators(num_qbits)['z']]

        # Ensure states are stored for coherence calculation
        temp_result = qt.mesolve(H_temp, create_ghz_state(num_qbits),
                                np.linspace(0, 10, 50), c_ops_temp, [], options=qt.Options(store_states=True))

        if temp_result.states: # Check if states were stored
            final_coh_metrics = calculate_quantum_coherence(temp_result.states[-1])
            final_coh = final_coh_metrics.get('l1_norm', 0) # Use get with default 0
        else:
            final_coh = 0
        final_coherences_vs_noise.append(final_coh)

    plt.plot(decoherence_rates, final_coherences_vs_noise, 'ro-', linewidth=2.5, markersize=6)
    plt.xlabel('Decoherence Rate')
    plt.ylabel('Final Coherence')
    plt.title('Decoherence Resilience')
    plt.grid(True, alpha=0.3)

    # 8. 3D visualization of material properties
    ax8 = plt.subplot(3, 3, 8, projection='3d')
    materials = list(barrier_properties.keys())
    x_vals = [barrier_properties[m]["gamma"] for m in materials]
    y_vals = [barrier_properties[m]["width"] for m in materials]
    z_vals = [barrier_properties[m]["topological_phase"] for m in materials]
    colors_3d = [barrier_properties[m]["decoherence"] for m in materials]

    scatter = ax8.scatter(x_vals, y_vals, z_vals, c=colors_3d, s=200, cmap='viridis', alpha=0.8)
    ax8.set_xlabel('Tunneling Coefficient (γ)')
    ax8.set_ylabel('Barrier Width (nm)')
    ax8.set_zlabel('Topological Phase')
    ax8.set_title('3D Material Properties')

    # Add material labels
    for i, mat in enumerate(materials):
        ax8.text(x_vals[i], y_vals[i], z_vals[i], f'  {mat}', fontsize=8)

    plt.colorbar(scatter, label='Decoherence Rate', shrink=0.5)

    # 9. Correlation matrix of observables
    ax9 = plt.subplot(3, 3, 9)

    if 'GHZ' in results and results['GHZ']['result'].expect: # Check if GHZ results and expectation values are available
        # Create correlation matrix from GHZ state observables
        ghz_expectations = np.array(results['GHZ']['result'].expect).T  # Transpose for time x observables
        if ghz_expectations.shape[0] > 1: # Ensure there is enough data for correlation
             correlation_matrix = np.corrcoef(ghz_expectations.T)

            # Create labels for observables
             obs_labels = [f'σₓ_{i+1}' for i in range(num_qbits)] + \
                         [f'σᵧ_{i+1}' for i in range(num_qbits)] + \
                         [f'σᵢ_{i+1}' for i in range(num_qbits)]

             # Only show first 6x6 for clarity
             if len(obs_labels) > 6:
                 correlation_matrix = correlation_matrix[:6, :6]
                 obs_labels = obs_labels[:6]

             im = plt.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
             plt.colorbar(im, label='Correlation')
             plt.xticks(range(len(obs_labels)), obs_labels, rotation=45)
             plt.yticks(range(len(obs_labels)), obs_labels)
             plt.title('Observable Correlations')
        else:
             ax9.text(0.5, 0.5, 'Not enough data for correlation matrix', horizontalalignment='center', verticalalignment='center', transform=ax9.transAxes)

    else:
        ax9.text(0.5, 0.5, 'GHZ Expectation values not available for Correlation Matrix', horizontalalignment='center', verticalalignment='center', transform=ax9.transAxes)


    plt.tight_layout()
    plt.savefig(f'comprehensive_quantum_analysis_{material_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Main execution
if __name__ == "__main__":
    print("🔬 Advanced Quantum Tunneling Simulation")
    print("=" * 50)

    # Run simulation for different materials
    for material in ["graphene", "boron_nitride"]:
        print(f"\n🧪 Analyzing {material}...")
        results, tlist, material_props = run_comprehensive_simulation(material)

        # Print summary statistics
        print(f"📊 Results for {material}:")
        for state_name, data in results.items():
            if data['coherence']: # Check if coherence data is available
                initial_coh = data['coherence'][0]
                final_coh = data['coherence'][-1]
                coherence_decay = (initial_coh - final_coh) / initial_coh * 100 if initial_coh != 0 else 0
                print(f"  {state_name}: Coherence decay = {coherence_decay:.1f}%")
            else:
                 print(f"  {state_name}: Coherence data not available")


        # Create comprehensive plots
        create_comprehensive_plots(results, tlist, material_props, material)

    print("\n✅ Simulation completed! Check the generated plots for detailed analysis.")
    print("\n📈 Key insights:")
    print("- GHZ states typically show faster decoherence but stronger initial entanglement")
    print("- Werner states provide better resilience to decoherence")
    print("- Graphene shows superior coherence preservation due to lower decoherence rates")
    print("- Topological phases contribute to non-trivial quantum correlations")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap

# Configurar estilo oscuro
plt.style.use('dark_background')

# 1. Definir las variables del sistema (tomadas del primer script)
x = -1
y = 0.5
z = 0
n = 1
w = -0.5

# 2. Definir la "cuantización" para los coeficientes (tomada del primer script)
c0 = (x * z)
c1 = (y * 2)
print(f"El coeficiente c0 calculado es: {c0}")
print(f"El coeficiente c1 calculado es: {c1}")

# 3. Definir los estados base (tomada del segundo script)
state_0 = np.array([[1], [0]], dtype=complex) # |0>
state_1 = np.array([[0], [1]], dtype=complex) # |1>

# 4. Construir y normalizar el estado cuántico (tomada del segundo script)
unnormalized_state = c0 * state_0 + c1 * state_1
norm = np.sqrt(np.abs(c0)**2 + np.abs(c1)**2)
if norm > 0:
    normalized_state = unnormalized_state / norm
else:
    normalized_state = state_1  # Estado |1> por defecto

print(f"Estado normalizado:\n{normalized_state}")

# 5. Calcular coordenadas de la esfera de Bloch (tomada del segundo script)
def state_to_bloch_coords(psi):
    """Convierte un estado cuántico a coordenadas de la esfera de Bloch"""
    c0, c1 = psi[0, 0], psi[1, 0]

    # Coordenadas de Bloch usando matrices de Pauli
    x = 2 * np.real(np.conj(c0) * c1)
    y = 2 * np.imag(np.conj(c0) * c1)
    z = np.abs(c0)**2 - np.abs(c1)**2

    return x, y, z

bloch_x, bloch_y, bloch_z = state_to_bloch_coords(normalized_state)

# Crear figura con subplots (tomada del segundo script)
fig = plt.figure(figsize=(16, 12))
fig.patch.set_facecolor('black')

# === VISUALIZACIÓN 1: ESFERA DE BLOCH === (tomada del segundo script)
ax1 = fig.add_subplot(221, projection='3d')
ax1.set_facecolor('black')

# Crear esfera
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

# Dibujar esfera translúcida
ax1.plot_surface(x_sphere, y_sphere, z_sphere,
                alpha=0.1, color='cyan', linewidth=0)

# Dibujar círculos de referencia
theta = np.linspace(0, 2*np.pi, 100)
# Ecuador
ax1.plot(np.cos(theta), np.sin(theta), 0, 'cyan', alpha=0.5, linewidth=1)
# Meridiano XZ
ax1.plot(np.cos(theta), 0, np.sin(theta), 'cyan', alpha=0.5, linewidth=1)
# Meridiano YZ
ax1.plot(0, np.cos(theta), np.sin(theta), 'cyan', alpha=0.5, linewidth=1)

# Ejes coordenados
ax1.quiver(0, 0, 0, 1.2, 0, 0, color='red', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 1.2, 0, color='green', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 0, 1.2, color='blue', alpha=0.8, arrow_length_ratio=0.1)

# Etiquetas de los ejes
ax1.text(1.3, 0, 0, 'X', color='red', fontsize=12)
ax1.text(0, 1.3, 0, 'Y', color='green', fontsize=12)
ax1.text(0, 0, 1.3, 'Z (|0⟩)', color='blue', fontsize=12)
ax1.text(0, 0, -1.3, '-Z (|1⟩)', color='blue', fontsize=12)

# Dibujar el estado cuántico como punto y vector
ax1.scatter([bloch_x], [bloch_y], [bloch_z],
           color='yellow', s=200, alpha=1, edgecolors='orange')
ax1.quiver(0, 0, 0, bloch_x, bloch_y, bloch_z,
          color='yellow', alpha=0.9, arrow_length_ratio=0.1, linewidth=3)

ax1.set_xlabel('X', color='white')
ax1.set_ylabel('Y', color='white')
ax1.set_zlabel('Z', color='white')
ax1.set_title('Esfera de Bloch\nEstado Cuántico |ψ⟩', color='white', fontsize=14)
ax1.set_xlim([-1.5, 1.5])
ax1.set_ylim([-1.5, 1.5])
ax1.set_zlim([-1.5, 1.5])

# === VISUALIZACIÓN 2: DISTRIBUCIÓN DE PROBABILIDAD === (tomada del segundo script)
ax2 = fig.add_subplot(222)
ax2.set_facecolor('black')

# Probabilidades de los estados base
prob_0 = np.abs(normalized_state[0, 0])**2
prob_1 = np.abs(normalized_state[1, 0])**2

states = ['|0⟩', '|1⟩']
probabilities = [prob_0, prob_1]
colors = ['lightblue', 'lightcoral']

bars = ax2.bar(states, probabilities, color=colors, alpha=0.7, edgecolor='white')
ax2.set_ylabel('Probabilidad', color='white')
ax2.set_title('Distribución de Probabilidad\nde Estados Base', color='white', fontsize=14)
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3)
ax2.tick_params(colors='white')

# Añadir valores en las barras
for bar, prob in zip(bars, probabilities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{prob:.3f}', ha='center', va='bottom', color='white', fontsize=12)

# === VISUALIZACIÓN 3: DENSIDAD DE DISTRIBUCIÓN 2D === (tomada del segundo script)
ax3 = fig.add_subplot(223)
ax3.set_facecolor('black')

# Crear una cuadrícula para la distribución de densidad
x_grid = np.linspace(-2, 2, 100)
y_grid = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x_grid, y_grid)

# Función de densidad gaussiana centrada en el estado
sigma = 0.5
density = np.exp(-((X - bloch_x)**2 + (Y - bloch_y)**2) / (2 * sigma**2))

# Colormap personalizado para fondo oscuro
colors_custom = ['black', 'darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red']
n_bins = 256
custom_cmap = LinearSegmentedColormap.from_list('custom', colors_custom, N=n_bins)

im = ax3.imshow(density, extent=[-2, 2, -2, 2], origin='lower',
                cmap=custom_cmap, alpha=0.8)
ax3.scatter([bloch_x], [bloch_y], color='white', s=100,
           marker='x', linewidth=3, label='Estado |ψ⟩')

ax3.set_xlabel('X', color='white')
ax3.set_ylabel('Y', color='white')
ax3.set_title('Densidad de Distribución 2D\n(Proyección XY)', color='white', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(colors='white')

# Colorbar
cbar = plt.colorbar(im, ax=ax3)
cbar.ax.yaxis.set_tick_params(color='white')
cbar.ax.yaxis.set_ticklabels(cbar.ax.yaxis.get_ticklabels(), color='green')

# === VISUALIZACIÓN 4: COMPONENTES COMPLEJAS === (tomada del segundo script)
ax4 = fig.add_subplot(224)
ax4.set_facecolor('black')

# Partes real e imaginaria de los coeficientes
real_parts = [np.real(normalized_state[0, 0]), np.real(normalized_state[1, 0])]
imag_parts = [np.imag(normalized_state[0, 0]), np.imag(normalized_state[1, 0])]

x_pos = np.arange(len(states))
width = 0.35

bars1 = ax4.bar(x_pos - width/2, real_parts, width, label='Parte Real',
                color='lightgreen', alpha=0.7, edgecolor='white')
bars2 = ax4.bar(x_pos + width/2, imag_parts, width, label='Parte Imaginaria',
                color='lightpink', alpha=0.7, edgecolor='white')

ax4.set_xlabel('Estados Base', color='white')
ax4.set_ylabel('Amplitud', color='white')
ax4.set_title('Componentes Complejas\nde los Coeficientes', color='white', fontsize=14)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(states)
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.tick_params(colors='white')

# Añadir valores en las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01 if height >= 0 else height - 0.05,
                f'{height:.3f}', ha='center', va='bottom' if height >= 0 else 'top',
                color='white', fontsize=10)

plt.tight_layout()
plt.show()

# Información adicional del estado (tomada del segundo script)
print(f"\n=== INFORMACIÓN DEL ESTADO CUÁNTICO ===")
print(f"Coordenadas de Bloch: ({bloch_x:.3f}, {bloch_y:.3f}, {bloch_z:.3f})")
print(f"Probabilidad |0⟩: {prob_0:.3f}")
print(f"Probabilidad |1⟩: {prob_1:.3f}")
print(f"Norma del estado: {np.sqrt(prob_0 + prob_1):.3f}")
print(f"Estado normalizado:\n{normalized_state}")

In [ ]:
import math

def wave_function(x_values, linear_phase=0.0):
    """
    Simula una función de onda 1D con un factor de fase lineal usando solo diccionarios.

    Args:
        x_values (list): Lista de coordenadas espaciales.
        linear_phase (float): Factor de fase lineal.

    Returns:
        dict: Diccionario con las coordenadas x como claves y los valores complejos como valores.
    """
    result = {}

    for x in x_values:
        # Función base: e^(-x²)
        psi_base = math.exp(-x**2)

        # Factor de fase: e^(i * linear_phase * x) = cos(linear_phase * x) + i * sin(linear_phase * x)
        phase_real = math.cos(linear_phase * x)
        phase_imag = math.sin(linear_phase * x)

        # Multiplicación de números complejos: psi_base * (phase_real + i * phase_imag)
        real_part = psi_base * phase_real
        imag_part = psi_base * phase_imag

        # Almacenar como tupla (parte_real, parte_imaginaria)
        result[x] = (real_part, imag_part)

    return result

# Función auxiliar para trabajar con números complejos como tuplas
def complex_magnitude(complex_tuple):
    """Calcula la magnitud de un número complejo representado como tupla."""
    real, imag = complex_tuple
    return math.sqrt(real**2 + imag**2)

def complex_phase(complex_tuple):
    """Calcula la fase de un número complejo representado como tupla."""
    real, imag = complex_tuple
    return math.atan2(imag, real)

# Ejemplo de uso
if __name__ == "__main__":
    # Coordenadas espaciales
    x_coords = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

    # Calcular función de onda con fase lineal
    wave_result = wave_function(x_coords, linear_phase=1.0)

    print("Función de onda calculada:")
    print("x\t\tReal\t\tImag\t\tMagnitud")
    print("-" * 50)

    for x, (real, imag) in wave_result.items():
        magnitude = complex_magnitude((real, imag))
        print(f"{x:4.1f}\t\t{real:8.4f}\t{imag:8.4f}\t{magnitude:8.4f}")

    # Ejemplo adicional: comparar con diferentes fases lineales
    print("\nComparación con diferentes fases lineales:")
    phases = [0.0, 0.5, 1.0, 2.0]

    for phase in phases:
        result = wave_function([0, 1], linear_phase=phase)
        print(f"Fase {phase}: x=0 → {result[0]}, x=1 → {result[1]}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

# 🔬 Representación del estado cuántico con modos Majorana
class MajoranaQubit:
    """
    Clase que representa un qubit cuántico, idealizado como un qubit de Majorana.
    """
    def __init__(self, initial_state=None):
        """
        Inicializa un qubit Majorana.

        Args:
            initial_state: Vector de estado inicial. Por defecto |0⟩.
        """
        if initial_state is None:
            # Estado base |0⟩ en notación de vector
            self.state = np.array([1, 0], dtype=complex)
        else:
            # Normalizar el estado inicial
            self.state = np.array(initial_state, dtype=complex)
            self.state = self.state / np.linalg.norm(self.state)

    def apply_gate(self, gate_matrix):
        """
        Aplica una puerta cuántica sobre el estado.

        Args:
            gate_matrix: Matriz 2x2 que representa la puerta cuántica.
        """
        self.state = np.dot(gate_matrix, self.state)
        # Renormalizar para evitar errores numéricos
        self.state = self.state / np.linalg.norm(self.state)

    def measure(self):
        """
        Realiza una medición del estado cuántico, causando el colapso.

        Returns:
            int: 0 o 1, resultado de la medición.
        """
        # Cálculo correcto de probabilidades
        p0 = np.abs(self.state[0])**2
        p1 = np.abs(self.state[1])**2
        result = np.random.choice([0, 1], p=[p0, p1])

        # Colapso del estado después de la medición
        self.state = np.array([1, 0], dtype=complex) if result == 0 else np.array([0, 1], dtype=complex)
        return result

    def get_bloch_vector(self):
        """
        Calcula las coordenadas del vector de Bloch para el estado actual.

        Returns:
            tuple: Coordenadas (x, y, z) en la esfera de Bloch.
        """
        # Extraer componentes del estado |psi⟩ = alpha|0⟩ + beta|1⟩
        alpha = self.state[0]
        beta = self.state[1]

        # Calcular componentes del vector de Bloch
        x = 2 * np.real(alpha * np.conj(beta))
        y = 2 * np.imag(alpha * np.conj(beta))
        z = np.abs(alpha)**2 - np.abs(beta)**2

        return (x, y, z)

    def get_density_matrix(self):
        """
        Calcula la matriz de densidad para el estado actual.

        Returns:
            array: Matriz de densidad 2x2.
        """
        return np.outer(self.state, np.conj(self.state))

# ⚡ Definimos puertas cuánticas comunes
class QuantumGates:
    """
    Colección de matrices que representan puertas cuánticas comunes.
    """
    # Matrices de Pauli
    PAULI_I = np.array([[1, 0], [0, 1]], dtype=complex)
    PAULI_X = np.array([[0, 1], [1, 0]], dtype=complex)
    PAULI_Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    PAULI_Z = np.array([[1, 0], [0, -1]], dtype=complex)

    # Otras puertas comunes
    HADAMARD = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)
    PHASE_S = np.array([[1, 0], [0, 1j]], dtype=complex)
    PHASE_T = np.array([[1, 0], [0, np.exp(1j * np.pi/4)]], dtype=complex)

    @staticmethod
    def rotation_x(theta):
        """Rotación alrededor del eje X en la esfera de Bloch."""
        return np.array([
            [np.cos(theta/2), -1j * np.sin(theta/2)],
            [-1j * np.sin(theta/2), np.cos(theta/2)]
        ], dtype=complex)

    @staticmethod
    def rotation_y(theta):
        """Rotación alrededor del eje Y en la esfera de Bloch."""
        return np.array([
            [np.cos(theta/2), -np.sin(theta/2)],
            [np.sin(theta/2), np.cos(theta/2)]
        ], dtype=complex)

    @staticmethod
    def rotation_z(theta):
        """Rotación alrededor del eje Z en la esfera de Bloch."""
        return np.array([
            [np.exp(-1j * theta/2), 0],
            [0, np.exp(1j * theta/2)]
        ], dtype=complex)

# 🚀 Simulación del sistema cuántico con microondas
class MicrowaveControl:
    """
    Clase para modelar la evolución dinámica de un qubit bajo control de microondas.
    """
    def __init__(self, frequency, amplitude=1.0, detuning=0.0):
        """
        Inicializa el control de microondas.

        Args:
            frequency: Frecuencia de la señal de microondas.
            amplitude: Amplitud de la señal (controla la fuerza del acoplamiento).
            detuning: Desajuste de la frecuencia respecto a la frecuencia de resonancia.
        """
        self.frequency = frequency
        self.amplitude = amplitude
        self.detuning = detuning
        self.time = 0.0

    def update_parameters(self, frequency=None, amplitude=None, detuning=None):
        """Actualiza los parámetros del control de microondas."""
        if frequency is not None:
            self.frequency = frequency
        if amplitude is not None:
            self.amplitude = amplitude
        if detuning is not None:
            self.detuning = detuning

    def modulate(self, qubit, dt=0.01):
        """
        Modula el qubit con microondas (control de fase en Majorana).

        Args:
            qubit: Instancia de MajoranaQubit a modular.
            dt: Incremento de tiempo para la evolución.
        """
        self.time += dt

        # Modulación realista con términos de fase y amplitud
        # Se puede entender como una rotación alrededor de un eje en el plano XY
        phase_shift = self.amplitude * np.sin(self.frequency * self.time * 2 * np.pi)
        detuning_effect = np.exp(1j * self.detuning * self.time)

        # Construimos el operador de evolución temporal U = exp(-i*H*t)
        # Aquí, el Hamiltoniano H se aproxima por términos de Pauli
        # que generan rotaciones.
        evolution_operator = np.array([
            [np.cos(phase_shift) * detuning_effect, -np.sin(phase_shift)],
            [np.sin(phase_shift), np.cos(phase_shift) * np.conj(detuning_effect)]
        ], dtype=complex)

        qubit.apply_gate(evolution_operator)

# 🔬 Simulación con evolución dinámica y entorno con ruido
class QuantumSimulation:
    """
    Clase que simula la evolución de un qubit bajo control y en presencia de ruido.
    """
    def __init__(self, qubit, microwave, decoherence_rate=0.01):
        """
        Inicializa la simulación cuántica.

        Args:
            qubit: Instancia de MajoranaQubit.
            microwave: Instancia de MicrowaveControl.
            decoherence_rate: Tasa de decoherencia (efectos del entorno).
        """
        self.qubit = qubit
        self.microwave = microwave
        self.decoherence_rate = decoherence_rate
        self.bloch_history = []

    def apply_decoherence(self):
        """Aplica efectos de decoherencia al estado cuántico."""
        # Implementación simplificada de decoherencia mediante un canal de despolarización
        if np.random.random() < self.decoherence_rate:
            # Elegir aleatoriamente entre las matrices de Pauli (X, Y, Z)
            pauli_matrices = [
                QuantumGates.PAULI_X,
                QuantumGates.PAULI_Y,
                QuantumGates.PAULI_Z
            ]
            # Elegir un índice aleatorio de la lista
            random_index = np.random.randint(0, len(pauli_matrices))
            # Aplicar la matriz seleccionada
            self.qubit.apply_gate(pauli_matrices[random_index])

    def run_step(self, apply_hadamard=True):
        """Ejecuta un paso de la simulación."""
        # Aplicamos el control de microondas
        self.microwave.modulate(self.qubit)

        # Aplicamos la puerta Hadamard para crear superposición
        if apply_hadamard:
            self.qubit.apply_gate(QuantumGates.HADAMARD)

        # Aplicamos la decoherencia
        self.apply_decoherence()

        # Registramos las coordenadas de Bloch
        self.bloch_history.append(self.qubit.get_bloch_vector())

    def run_simulation(self, steps, apply_hadamard=True):
        """
        Ejecuta la simulación completa.

        Args:
            steps: Número de pasos a simular.
            apply_hadamard: Si es True, aplica la puerta Hadamard en cada paso.
        """
        for _ in range(steps):
            self.run_step(apply_hadamard)

        return self.bloch_history

# 🎯 Función para visualización avanzada
def animate_bloch_sphere(simulation, title="Simulación Cuántica Animada"):
    """
    Crea una animación de la trayectoria en la esfera de Bloch.

    Args:
        simulation: Instancia de QuantumSimulation.
        title: Título para la animación.
    """
    # Configurar estilo oscuro
    plt.style.use('dark_background')

    fig = plt.figure(figsize=(10, 10))
    fig.patch.set_facecolor('black')
    ax = fig.add_subplot(111, projection='3d')
    ax.set_facecolor('black')

    # Configurar la esfera de Bloch
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x_sphere = np.cos(u) * np.sin(v)
    y_sphere = np.sin(u) * np.sin(v)
    z_sphere = np.cos(v)
    ax.plot_wireframe(x_sphere, y_sphere, z_sphere, color="cyan", alpha=0.3)

    # Dibujar los ejes de la esfera de Bloch
    ax.quiver(0, 0, 0, 1, 0, 0, color='r', length=1.3, arrow_length_ratio=0.1, alpha=0.8)
    ax.quiver(0, 0, 0, 0, 1, 0, color='g', length=1.3, arrow_length_ratio=0.1, alpha=0.8)
    ax.quiver(0, 0, 0, 0, 0, 1, color='b', length=1.3, arrow_length_ratio=0.1, alpha=0.8)
    ax.text(1.4, 0, 0, 'X', color='r', fontsize=12)
    ax.text(0, 1.4, 0, 'Y', color='g', fontsize=12)
    ax.text(0, 0, 1.4, '|0⟩ (Z)', color='b', fontsize=12)
    ax.text(0, 0, -1.4, '|1⟩ (-Z)', color='b', fontsize=12)
    ax.set_title(title, color='white', fontsize=16)

    # Inicializar la línea de la trayectoria
    line, = ax.plot([], [], [], 'yellow', lw=3, label='Trayectoria del estado', alpha=0.8)
    # Inicializar el punto que representa el estado actual
    point, = ax.plot([], [], [], 'o', color='orange', markersize=12)

    # Límites de los ejes
    ax.set_xlim([-1.5, 1.5])
    ax.set_ylim([-1.5, 1.5])
    ax.set_zlim([-1.5, 1.5])
    ax.set_xlabel('X', color='white')
    ax.set_ylabel('Y', color='white')
    ax.set_zlabel('Z', color='white')

    def update(frame):
        """Función de actualización para la animación."""
        simulation.run_step(apply_hadamard=False)
        history = np.array(simulation.bloch_history)

        if history.shape[0] > 0:
            x_coords = history[:, 0]
            y_coords = history[:, 1]
            z_coords = history[:, 2]

            line.set_data(x_coords, y_coords)
            line.set_3d_properties(z_coords)

            point.set_data([x_coords[-1]], [y_coords[-1]])
            point.set_3d_properties([z_coords[-1]])

        return line, point

    # Crear y devolver la animación
    ani = FuncAnimation(fig, update, frames=300, interval=50, blit=True, repeat=True)
    return ani

def plot_static_analysis(simulation, steps=100):
    """
    Crea un análisis estático de la simulación con múltiples visualizaciones.
    """
    # Ejecutar simulación
    history = simulation.run_simulation(steps, apply_hadamard=True)
    history = np.array(history)

    # Configurar estilo oscuro
    plt.style.use('dark_background')

    fig = plt.figure(figsize=(15, 10))
    fig.patch.set_facecolor('black')

    # 1. Esfera de Bloch 3D
    ax1 = fig.add_subplot(221, projection='3d')
    ax1.set_facecolor('black')

    # Dibujar esfera
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
    ax1.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.1, color='cyan')

    # Trayectoria
    ax1.plot(history[:, 0], history[:, 1], history[:, 2], 'yellow', linewidth=2, alpha=0.8)
    ax1.scatter(history[-1, 0], history[-1, 1], history[-1, 2],
               color='orange', s=100, alpha=1)
    ax1.set_title('Trayectoria en Esfera de Bloch', color='white')
    ax1.set_xlabel('X', color='white')
    ax1.set_ylabel('Y', color='white')
    ax1.set_zlabel('Z', color='white')

    # 2. Evolución temporal de coordenadas
    ax2 = fig.add_subplot(222)
    ax2.set_facecolor('black')
    time_steps = np.arange(len(history))
    ax2.plot(time_steps, history[:, 0], 'r-', label='X', alpha=0.8)
    ax2.plot(time_steps, history[:, 1], 'g-', label='Y', alpha=0.8)
    ax2.plot(time_steps, history[:, 2], 'b-', label='Z', alpha=0.8)
    ax2.set_xlabel('Paso de tiempo', color='white')
    ax2.set_ylabel('Coordenadas de Bloch', color='white')
    ax2.set_title('Evolución Temporal', color='white')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(colors='white')

    # 3. Distribución de coordenadas
    ax3 = fig.add_subplot(223)
    ax3.set_facecolor('black')
    ax3.hist(history[:, 0], bins=20, alpha=0.5, color='red', label='X')
    ax3.hist(history[:, 1], bins=20, alpha=0.5, color='green', label='Y')
    ax3.hist(history[:, 2], bins=20, alpha=0.5, color='blue', label='Z')
    ax3.set_xlabel('Valor de coordenada', color='white')
    ax3.set_ylabel('Frecuencia', color='white')
    ax3.set_title('Distribución de Coordenadas', color='white')
    ax3.legend()
    ax3.tick_params(colors='white')

    # 4. Análisis de coherencia
    ax4 = fig.add_subplot(224)
    ax4.set_facecolor('black')
    # Calcular la norma del vector de Bloch (medida de coherencia)
    coherence = np.sqrt(history[:, 0]**2 + history[:, 1]**2 + history[:, 2]**2)
    ax4.plot(time_steps, coherence, 'magenta', linewidth=2, alpha=0.8)
    ax4.set_xlabel('Paso de tiempo', color='white')
    ax4.set_ylabel('Coherencia |r|', color='white')
    ax4.set_title('Evolución de la Coherencia', color='white')
    ax4.grid(True, alpha=0.3)
    ax4.tick_params(colors='white')
    ax4.set_ylim([0, 1.1])

    plt.tight_layout()
    return fig

def main():
    """
    Función principal que ejecuta diferentes tipos de simulaciones y visualizaciones.
    """
    print("🔬 Simulación de Qubit Majorana")
    print("=" * 50)

    # Inicializar el sistema cuántico
    print("Inicializando qubit Majorana...")
    qubit = MajoranaQubit()
    print(f"Estado inicial: {qubit.state}")
    print(f"Vector de Bloch inicial: {qubit.get_bloch_vector()}")

    # Configurar control de microondas
    print("\nConfigurando control de microondas...")
    microwave = MicrowaveControl(frequency=0.5, amplitude=0.2, detuning=0.05)
    print(f"Frecuencia: {microwave.frequency} Hz")
    print(f"Amplitud: {microwave.amplitude}")
    print(f"Desajuste: {microwave.detuning}")

    # Configurar simulación
    print("\nConfigurando simulación...")
    simulation = QuantumSimulation(qubit, microwave, decoherence_rate=0.005)
    print(f"Tasa de decoherencia: {simulation.decoherence_rate}")

    # Ejecutar análisis estático
    print("\n📊 Generando análisis estático...")
    static_fig = plot_static_analysis(simulation, steps=200)

    # Crear nueva simulación para la animación
    print("\n🎬 Creando animación de la esfera de Bloch...")
    qubit_anim = MajoranaQubit()
    microwave_anim = MicrowaveControl(frequency=0.8, amplitude=0.15, detuning=0.02)
    simulation_anim = QuantumSimulation(qubit_anim, microwave_anim, decoherence_rate=0.002)

    # Crear animación
    animation = animate_bloch_sphere(simulation_anim,
                                   title="Evolución Dinámica del Qubit Majorana")

    print("\n✅ Simulación completada!")
    print("Mostrando visualizaciones...")

    # Mostrar resultados
    plt.show()

    # Demostración de mediciones
    print("\n🎯 Demostrando mediciones cuánticas:")
    test_qubit = MajoranaQubit()
    test_qubit.apply_gate(QuantumGates.HADAMARD)  # Crear superposición
    print(f"Estado en superposición: {test_qubit.state}")
    print(f"Vector de Bloch: {test_qubit.get_bloch_vector()}")

    # Realizar varias mediciones
    measurements = []
    for i in range(10):
        test_qubit_copy = MajoranaQubit(test_qubit.state)
        result = test_qubit_copy.measure()
        measurements.append(result)

    print(f"Resultados de 10 mediciones: {measurements}")
    print(f"Probabilidad de |0⟩: {measurements.count(0)/len(measurements):.2f}")
    print(f"Probabilidad de |1⟩: {measurements.count(1)/len(measurements):.2f}")

    return animation, static_fig

if __name__ == "__main__":
    # Ejecutar la simulación principal
    animation, static_figure = main()

    # Mantener las ventanas abiertas
    input("\nPresiona Enter para finalizar el programa...")